In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
#Maximum Surface Excess Concentration Prediction
def predict_Gamma_max(MaxSEC_filepath):     #MaxSEC_filepath is the file path to the AlvaDesc descriptors used to 
                                            #predict the maximum surface excess concentration (in the order shown in the SI)
    X_pred_Gamma_max = pd.read_excel(MaxSEC_filepath)
    estimator_MaxSEC = pickle.load(open('MaxSEC-model.sav', 'rb'))
    y_pred_Gamma_max = estimator_MaxSEC.predict(X_pred_Gamma_max)
    y_pred_Gamma_max *= 10**-6
    return(y_pred_Gamma_max)

#logKL Prediction
def predict_KL(KL_filepath):          #KL_filepath is the file path to the SIRMS descriptors used to 
                                      #predict KL (in the order shown in the SI)
    X_pred_logKL = pd.read_excel(KL_filepath)
    estimator_logKL = pickle.load(open('logKL-model.sav', 'rb'))
    y_pred_logKL = estimator_logKL.predict(X_pred_logKL)
    KL = 10**y_pred_logKL
    return(KL)

#logCMC Prediction
def predict_CMC(CMC_filepath):        #CMC_filepath is the file path to the CDK descriptors used to 
                                      #predict CMC (in the order shown in the SI)
    X_pred_logCMC = pd.read_excel(CMC_filepath)
    estimator_logCMC = pickle.load(open('logCMC-model.sav', 'rb'))
    y_pred_logCMC = estimator_logCMC.predict(X_pred_logCMC)
    CMC = 10**y_pred_logCMC
    return(CMC)

#Prediction for All
def predict_all(MaxSEC_filepath, logKL_filepath, logCMC_filepath):
    MaxSEC_list = predict_Gamma_max(MaxSEC_filepath)
    KL_list = predict_KL(KL_filepath)
    CMC_list = predict_CMC(CMC_filepath)
    
    for i, _ in enumerate(MaxSEC_list):
        print(MaxSEC_list[i], KL_list[i], CMC_list[i])
        
        
        
        

#Plots the SFT-log(c) isotherm(s) for surfactants. MaxSEC_filepath is the file path to the AlvaDesc descriptors used to 
#predict the maximum surface excess concentration, logKL_filepath is the file path to the SIRMS descriptors used to 
#predict KL, logCMC_filepath is the file path to the CDK descriptors used to predict CMC, savedir is the figure directory
#you wish to save in, alltogether is used to plot the surfactant isotherms all on one graph, type 'alltogether=True' if
#you want them plotted together, otherwise ignore. These isotherms use the Szyszkowski equation.
def predict_isotherm(MaxSEC_filepath, KL_filepath, CMC_filepath, savedir, alltogether=False):
    MaxSEC_list = predict_Gamma_max(MaxSEC_filepath)
    KL_list = predict_KL(KL_filepath)
    CMC_list = predict_CMC(CMC_filepath)
    colors = plt.cm.Blues(np.linspace(0.4,1,len(MaxSEC_list)))
    
    xfine = np.geomspace(1e-10, 10, 100000)
    logxfine = np.log10(xfine)
    
    for i, _ in enumerate(MaxSEC_list):
        print(MaxSEC_list[i], KL_list[i], CMC_list[i])
        MaxSEC = MaxSEC_list[i]
        KL = KL_list[i]
        CMC = CMC_list[i]
        
        yfine = []

        for j, _ in enumerate(xfine):
            if xfine[j] < CMC:
                yfinei = 72-(8314 * 298.15 * MaxSEC * np.log(1 + (KL*xfine[j]*1000)))
                yfine.append(yfinei)
            else:
                yfinei = 72-(8314 * 298.15 * MaxSEC * np.log(1 + (KL*CMC*1000)))
                yfine.append(yfinei)
                
        if alltogether==True:
            plt.plot(logxfine, yfine, color=colors[i], label='Surfactant '+str(i+1))
            plt.title('All Surfactants')
            plt.xlabel('log(c)')
            plt.ylabel('SFT (mN/m)')
            plt.legend()
            
            plt.savefig(savedir+'/AllSurfactants.jpg', dpi=1500, bbox_inches='tight')
        else:
            plt.plot(logxfine, yfine, 'b')
            plt.title('Surfactant '+str(i+1))
            plt.xlabel('log(c)')
            plt.ylabel('SFT (mN/m)')
            
            
            plt.savefig(savedir+'/Surfactant '+str(i+1)+'.jpg', dpi=1500, bbox_inches='tight')
            plt.clf()

In [4]:
# To start predicting, simply type 'predict_isotherm(MaxSEC_filepath, KL_filepath, CMC_filepath, savedir)'
# with your descriptor filepaths and the folder you wish to save figures within.